#  NeuroSpin tutorial, subject-level encoding models (beginner level)
__Content creator:__ Florent Meyniel, NeuroSpin, CEA Paris-Saclay

I would like to acknowledge my colleagues [Le Ster _et al._ (2019)](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0225286) who shared their data on the OpenNeuro platform ([here](https://openneuro.org/datasets/ds002606)). This notebook analyzes a sample participant from this dataset.

Note that for the purpose of speeding up computations in this tutorial, I have resampled the data to a coarse spatial resolution (4 mm isotropic), but the original data where collected at a beautiful 1.6 mm isotropic resolution on a 7T scanner.


## Set-up the environment

The following cell installs the libraries that we will use.

In [ ]:
# install libraries
! pip install -U nilearn

This cell downloads the data that we need.

In [ ]:
# clone the git repository to get the data
! git clone https://github.com/florentmeyniel/cogmaster_neuro102.git

Wait until the download is complete. When you see the folder:
>cogmaster_neuro102

in the directory (in the left panel), you can continue and execute the next cell to move in the folder containing the data.

In [ ]:
cd cogmaster_neuro102/

Now, do various imports.

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm 

from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
from nilearn.glm.first_level import run_glm
from nilearn.glm import compute_contrast
from nilearn.glm import fdr_threshold
from nilearn import plotting
from nilearn.maskers import NiftiMasker
from nilearn import surface
from nilearn import datasets

%matplotlib inline

---
# Part 1: Regression analysis and the General Linear Model (GLM)

## Example data set: A simple functional "localizer"

The data we are going to analyze were collected during a "localizer" paradigm, developped by [P. Pinel et al, BMC 2007](https://bmcneurosci.biomedcentral.com/articles/10.1186/1471-2202-8-91).


Here is (a portion of) the abstract of this paper:
>Although cognitive processes such as reading and calculation are associated with reproducible cerebral networks, inter-individual variability is considerable. Understanding the origins of this variability will require the elaboration of large multimodal databases compiling behavioral, anatomical, genetic and functional neuroimaging data over hundreds of subjects. With this goal in mind, we designed a simple and fast acquisition procedure based on a 5-minute functional magnetic resonance imaging (fMRI) sequence that can be run as easily and as systematically as an anatomical scan, and is therefore used in every subject undergoing fMRI in our laboratory. This protocol captures the cerebral bases of auditory and visual perception, motor actions, reading, language comprehension and mental calculation at an individual level.

Here is a figure of the paper that describes the task:
![pinel](https://media.springernature.com/full/springer-static/image/art%3A10.1186%2F1471-2202-8-91/MediaObjects/12868_2007_Article_389_Fig1_HTML.jpg)

## Building a design matrix
We will build a design matrix that corresponds to this task. The following file lists the events and their timing in the task.

Execute the code that loads the data and browse its content.  
**Q1-1: Given the label of trials, is this design amenable to categorical or parametric regressors?**

In [ ]:
# load event file
task_file = 'sub-01_ses-01_locAP-sms-1-6iso-events.tsv'
events = pd.read_table(task_file)
events

The code below will construct a design matrix for this task.  
Note that the ten first columns correspond to task-related regressors.  
**Q1-2: Explain how those regressors are constructed**  

The design matrix also has columns mvt0 to mvt5 corresponding to the volume-to-volume movements of subjects (3 parameters for translation along x, y, z axis, and 3 parameters for rotation pitch, roll, yaw), and drift parameters (drift_1 to drift_4).  
**Q1-3: Why do we include those extra regressors in the design matrix?**

In [ ]:
# load movement parameters
movement = pd.read_csv('rp_asub-01_locAP_multiband_1_6iso.txt', sep='\s+',
                       header=None, names=[f"mvt{k}" for k in range(6)])

# get frame times (when each frame, a.k.a. volume, is collected)
TR = 1.2285
frame_times = np.arange(movement.shape[0])*TR

# make design matrix
design_matrix = make_first_level_design_matrix(frame_times,
                                               events, 
                                               drift_model='cosine',
                                               high_pass=1/128,
                                               add_regs=movement,
                                               add_reg_names=[name for name in movement.columns])
plot_design_matrix(design_matrix)

The following cell of code will select one column: the "audio_left_hand" column. The blue curve is the predicted fMRI timecourse elicited by this stimulus, and the dots denote the exact timing of those events.

In [ ]:
plt.plot(frame_times, design_matrix['audio_left_hand'], label='predicted BOLD')
plt.plot(events.loc[events['trial_type']=='audio_left_hand', 'onset'].values, 
         [0]*len(events.loc[events['trial_type']=='audio_left_hand', 'onset'].values),
         'o', label='left click, audio instruction')
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Signal')

**Q2-4: We often say that the fMRI response is indirect, delayed and slow. Why?**  

**Q2-5: The last peak is nearly twice as large as the other peaks. Why?**  
Hint: you may want to zoom onto what happened around 250s in the task with the cell below.

In [ ]:
events[(events["onset"]>250) & (events["onset"]<265)]

## Analysis of one voxel
We are now going to analyze the activity of a single example voxel. First, load the data. 

In [ ]:
# load data
masker = NiftiMasker(mask_strategy='epi',
                    detrend=True,
                    high_pass=1/128,
                    t_r=TR)
fMRI_data = masker.fit_transform('swtrasub-01_locAP_multiband_4mm.nii')

Now extract an example voxel and plot its activity.  
**Q1-6: Describe the timeseries**

In [ ]:
# let's focus on one voxel
voxel_index = 4999
example_voxel = fMRI_data[:, voxel_index]

# and plot it's activity
plt.plot(frame_times, fMRI_data[:, voxel_index])
plt.xlabel('Time (s)')
plt.ylabel('BOLD signal')

The following cell will compute and plot the predicted fMRI signal triggered by three types of events:
- a click with the left hand
- a click with the right hand
- flashing a checkerboard

In [ ]:
# Plot prediction signal
predicted = {'left_click': design_matrix['audio_left_hand'].values + design_matrix['video_left_hand'].values,
             'right_click':design_matrix['audio_right_hand'].values + design_matrix['video_right_hand'].values,
              'checkerboard': design_matrix['horizontal_checkerboard'].values + \
             design_matrix['vertical_checkerboard'].values}

for k, stimulus in enumerate(predicted):
    plt.subplot(3,1,k+1)
    plt.plot(frame_times, predicted[stimulus])
    plt.xlabel('Time (s)')
    plt.ylabel('BOLD signal')
    plt.title(stimulus)
plt.tight_layout()

**Q2-7: Compare visually the observed timeseries and those three predicted timeseries. Which one is most similar to the observed timeseries?**  

The following cell plots the observed and predicted signal one against the other.  
**Q2-8: Interpret the X-Y plot below**

In [ ]:
# plot predicted vs. observed
for k, stimulus in enumerate(predicted):
    plt.subplot(1,3,k+1)
    plt.plot(predicted[stimulus], example_voxel, '.')
    plt.xlabel('Predicted')
    plt.ylabel('Observed')
    plt.title(stimulus)
plt.tight_layout()

**Q1-9 What is a regression weight (a.k.a. "beta")?**  
The cell below computes and plots the (ordinary least square) estimate of the regression weights.  
Note that I use the formula below. In practice, this is packaged in your analysis software (as we will see later).

In [ ]:
# Normalize regressors
X = design_matrix.values
Xz = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
Xz[:,-1] = 1

# Compute the regression weights
beta = np.linalg.inv(Xz.T @ Xz) @ Xz.T @ example_voxel
plt.bar(np.arange(len(beta)), beta)
plt.xticks(ticks=np.arange(len(beta)),
          labels=[name for name in design_matrix.columns],
          rotation=60, ha='right')
plt.ylabel('beta')

The regressors have been normalized so that the magnitude of the betas can be compared among each other.  
**Q1-10: Interpret the betas. What appears to be a likely cause of the activity in this voxel?**  

In the course, we saw that we can test effects in the data with contrasts. Below I specify two example contrasts:
- "left click elicits more signal than right click" (displayed)
- "audio stimuli elicits more signal than visual stimuli" (not displayed below).  

**Q1-11: How is a contrast constructed?**

In [ ]:
# Specify contrast
contrasts = {
    'left_hand': np.array([1 if 'left_hand' in name else 0 for name in design_matrix.columns]),
    'right_hand': np.array([1 if 'right_hand' in name else 0 for name in design_matrix.columns]),
    'audio': np.array([1 if 'audio' in name else 0 for name in design_matrix.columns]),
    'video': np.array([1 if 'video' in name else 0 for name in design_matrix.columns]),}
contrasts['left - right click'] = contrasts['left_hand'] - contrasts['right_hand']
contrasts['audio - video'] = contrasts['audio'] - contrasts['video']

# Show contrast
print(contrasts['left - right click'])
plt.bar(np.arange(len(contrasts['left - right click'])),
        contrasts['left - right click'])
plt.xticks(ticks=np.arange(len(contrasts['left - right click'])),
          labels=[name for name in design_matrix.columns],
          rotation=60, ha='right')
plt.title('Contrast: '+ 'left - right click')

Given the name of the columns in the design matrix, **Q1-12: What does the following contrast correspond to?**
[1 0 0 -1 0 0 1 0 0 -1 0 0 0 0 0 0 0 0 0 0 0]  

Below I compute the t-value corresponding to the two contrasts "left - right click" and "audio - video".  
**Q1-13: Which t-value is the largest? Interpret**  

Note that I use the formula for educational purpose. This computation is also packaged in standard analysis softwares. 

In [ ]:
# compute the t-value of two contrast
def compute_tvalue(y, X, beta, c):
    c = c[:, np.newaxis]
    res = y - X @ beta
    tval = c.T @ beta / (np.std(res) * np.sqrt(c.T @ np.linalg.inv(X.T @ X) @ c))
    return tval[0][0]

for contrast_id in ['left - right click', 'audio - video']:
    print(contrast_id, ', tvalue=',
          compute_tvalue(example_voxel, Xz, beta,
                         contrasts[contrast_id]))

If time permits, you can rerun the cells above, starting from "# let's focus on one voxel" where you change the index of the voxel (the original value is 4999, you can pick any value from 0 to 26942, which is the number of voxels in this data set). Execute the cells in the order in which they appear.

In pratice, one does not compute the beta weights and tvalues by hand like I did here, but instead uses existing packages.
For instance here is a [regression function](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) available in scikit-learn that returns the beta estimates.
And another regression function that returns the beta estimates and their statistics (t-value, p-value, etc.) can be found in [statsmodel](https://www.statsmodels.org/stable/regression.html).
Specific packages exist for the analysis of fMRI data, such as [linear](nilearn.github.io/) that we use below.

---
# Part 2: Whole brain analysis of the "localizer" with encoding models  

## Estimation and contrast: Testing effects

We are now going to analyze all voxels, not just an example.  
**Q2-1: How does one call this type of regression which is repeated for every voxel?**

We are going to use the same contrasts introduced above (plotted below).

In [ ]:
# Plot the contrasts
for k, contrast_id in enumerate(['left - right click', 'audio - video']):
    plt.subplot(2,1,k+1)
    plt.bar(np.arange(len(contrasts[contrast_id])),
            contrasts[contrast_id])
    plt.plot([0, len(contrasts[contrast_id])-1], [0, 0], 'k')
    plt.xlim(-0.5, len(contrasts[contrast_id])-0.5)
    plt.title(contrast_id)
plt.xticks(ticks=np.arange(len(contrasts['left - right click'])),
          labels=[name for name in design_matrix.columns],
          rotation=60, ha='right')
plt.tight_layout()

The following line of code uses a function from nilearn, a Python package, to estimate the general linear model (GLM) on all voxels.

In [ ]:
# Estimate GLM on all voxels
labels, estimates = run_glm(fMRI_data, design_matrix.values)

The function run_glm does actually a bit more than the simple ordinary least square (OLS) solution used above with the formula. Check the help of the function (by running the next cell). It uses the option with ar1, which stands fro auto-regressive model.  
**Q2-2 Why do we use an auto-regressive model for the estimation?**  
Note that the OLS can also be used (but it is not the default option of run_glm).

In [ ]:
run_glm?

To illustrated the temporal autocorrelation of the signal, we can plot the autocorrelation function for an example voxel.  
**Q2-3 What is the autocorrelation in this voxel (correlation between successive values)?**  
NB: What is problematic for regression model is actually the autocorrelation of the residual timeseries

In [ ]:
# compute and plot the autocorrelation function of an example voxel
TR_range = range(1, 20)
auto_correlation = [np.corrcoef(example_voxel[:-lag], example_voxel[lag:])[0, 1]
                    for lag in TR_range]
plt.plot(TR_range, auto_correlation, '-o')
plt.ylabel('correlation')
plt.xlabel('lag (# scans)')
plt.grid(visible=True)

The following cell computes the t-value of the contrast. This is similar to the computation done by hand above.

In [ ]:
# Estimate contrast
contrast_id = 'left - right click'
contrast = compute_contrast(labels, estimates,
                            contrasts[contrast_id],
                            contrast_type='t')
t_val = masker.inverse_transform(contrast.stat())

We are now going to display the thresholded T-map.

In [ ]:
# Plot glass brain
plotting.plot_glass_brain(t_val, threshold=3, plot_abs=False,
                          colorbar=True, title=contrast_id)

**Q2-4 What is a glass brain representation?**  
The following cell renders the results on slices.

In [ ]:
# Plot slices
plotting.plot_stat_map(t_val, display_mode='z',
                       threshold=3.0, title=contrast_id,
                       cut_coords=8)

**Q2-5: What is the orientation of the slices presented above?**  

The following cell renders the results on the surface of the right hemisphere.

In [ ]:
# get canonical surface anatomy
fsaverage = datasets.fetch_surf_fsaverage()

# project the results onto the mesh of cortical surface
texture = surface.vol_to_surf(t_val, fsaverage.pial_right)

# plot
plotting.plot_surf_stat_map(
        fsaverage.infl_right, texture, hemi='right',
        title=contrast_id, colorbar=True,
        threshold=3.0, bg_map=fsaverage.sulc_right)

**Q2-6: What do the blue-ish and red-ish blobs correspond to?**  
**Q2-7: Interpret the results.**  

The next cell now estimates and reports the "audio-visual" contrast.

In [ ]:
# Estimate contrast
contrast_id = 'audio - video'
contrast_AV = compute_contrast(labels, estimates,
                            contrasts[contrast_id],
                            contrast_type='t')
t_val_AV = masker.inverse_transform(contrast_AV.stat())
plotting.plot_glass_brain(t_val_AV, threshold=3, plot_abs=False,
                          colorbar=True, title=contrast_id)

**Q2-8: Interpret the results.**  

## Comparison of different noise models
The above estimation and inference assumed autocorrelated noise with an AR one model.  
Let's compare the results with the OLS estimates.

In [ ]:
# Estimate GLM on all voxels
labels_OLS, estimates_OLS = run_glm(fMRI_data, design_matrix.values, noise_model='ols')

# Estimate contrast
contrast_id = 'left - right click'
contrast_OLS = compute_contrast(labels_OLS, estimates_OLS,
                            contrasts[contrast_id],
                            contrast_type='t')
t_val_OLS = masker.inverse_transform(contrast_OLS.stat())

# Plot results
plotting.plot_glass_brain(t_val_OLS, threshold=3, plot_abs=False,
                          colorbar=True, title=contrast_id)

In [ ]:
plt.plot(contrast.stat(), contrast_OLS.stat(), '.')
plt.plot([-12, 15], [-12, 15], 'k')
plt.xlabel('AR1 estimate')
plt.ylabel('OLS estimate')
plt.title(f"T-values")
plt.grid('on')

**Q2.9 Significance with OLS estimates is biased. In which direction? Why?**   

Let's know compare the beta estimates themselves.

In [ ]:
plt.plot(contrast.effect_size(), contrast_OLS.effect_size(), '.')
plt.plot([-100, 200], [-100, 200], 'k')
plt.xlabel('AR1 estimate')
plt.ylabel('OLS estimate')
plt.title('Beta-values')
plt.grid('on')

**Q2.10 Are OLS estimates of beta weights biased? Should we care about the noise model for group-level inference?**   


## Statistical significance and correction for multiple comparisons
The figure above is displayed with a threshold of +/- 3.  
**Q3-10: Change the code to set a more liberal and a more conservative threshold.**  
**Q3-11: Do you think that some voxels are actually false positives? Why?**  

We are going to use parametric statistics (i.e. the assumption that the t-values follow a known distribution) to assess significance levels as p-values.  
The following cell shows the map thresholded at p<0.001 (two-sided test, controlling for both positive and negative effects).  
Technical detail: the t-values are transformed into z-values because it is more convenient to do so with the nilearn package.  
The z-value corresponding to p<0.001 (two-sided test) uncorrected is displayed.

In [ ]:
z_vals = contrast_AV.z_score()
z_map = masker.inverse_transform(z_vals)
alpha_threshold = 0.001
z_thd = norm.isf(alpha_threshold/2)
print(f"FPR threshold (p={alpha_threshold}, two-sided test): z={z_thd:0.2f}")
plotting.plot_glass_brain(z_map, threshold=z_thd, plot_abs=False,
                          colorbar=True, title=contrast_id+' uncorr.')

The next cell computes a new threshold that controls for a given false discovery rate (FDR). This estimation is done with the Benjamini-Hochberg procedure. For details on this procedure, and more generally, an introduction to multiple comparisons correction, see this [review by T. Nichols and S. Hayasaka, 2003](https://journals.sagepub.com/doi/pdf/10.1191/0962280203sm341ra).   
The threshold (z-value) that corrects for the false discovery rate at p<0.001 (two-sided test) is reported and used to threshold the map.  
**Q2-11: Interpret the difference between corrected and uncorrected maps**  

In [ ]:
fdr_z_thd = fdr_threshold(np.abs(z_vals), alpha_threshold/2)
print(f"FDR threshold (p={alpha_threshold}, two-sided test): z={fdr_z_thd:0.2f}")
plotting.plot_glass_brain(z_map, threshold=fdr_z_thd, plot_abs=False,
                          colorbar=True, title=contrast_id+' corr.')

FDR is a weak correction. See the mentioned-aboved review for more conservative corrections. 
The family-wise error rate (FWER) correction is more conservative. When samples are assumed to be independent, the FWER correction is known as the Bonferroni correction. This correction is overly conservative in fMRI because the signals are spatially smooth (thus, not independent).
Here is the result of the Bonferroni correction of our contrast of interest:

In [ ]:
alpha_threshold = 0.05
fwe_z_thd = norm.isf((alpha_threshold/2)/z_vals.shape[0])
print(f"FWE (Bonferroni) threshold (p={alpha_threshold}, two-sided test): z={fwe_z_thd:0.2f}")
plotting.plot_glass_brain(z_map, threshold=fwe_z_thd, plot_abs=False,
                          colorbar=True, title=contrast_id+' corr.')

In nilearn, you can use nilearn.glm.threshold_stats_img (with the option 'fpr' for uncorrected, 'fdr' and 'bonferroni') to perform those corrections, and exclude clusters smaller than a given threshold. Here is an example:

In [ ]:
from nilearn.glm import threshold_stats_img
fwer_map_without_small_clusters, fwer_z_thd = threshold_stats_img(
    z_map, alpha=alpha_threshold, height_control='bonferroni', cluster_threshold=50)
print(f"FWE (Bonferroni) threshold (p={alpha_threshold}, two-sided test): z={fwe_z_thd:0.2f}")
plotting.plot_glass_brain(fwer_map_without_small_clusters,
                          threshold=fwer_z_thd, plot_abs=False,
                          colorbar=True, title=contrast_id+' corr.')